In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
spark = SparkSession.builder \
    .appName("Smart Supply Chain Analytics") \
    .getOrCreate()

print(spark.version)

4.0.0


# List Dataset Files

In [4]:
import os

In [5]:
print(os.listdir("."))

['.ipynb_checkpoints', 'artifacts', 'DataCo-Supply-Chain-Analysis.zip', 'DataCoSupplyChainDataset.csv', 'DescriptionDataCoSupplyChain.csv', 'Smart-Supply-Chain-Analytics.ipynb', 'tokenized_access_logs.csv']


# Read File

In [6]:
import pandas as pd
pd.set_option("display.max_columns", None)

In [7]:
dataco = spark.read.option("header", True) \
    .option("inferSchema", True) \
    .option("encoding", "iso-8859-1") \
    .csv("DataCoSupplyChainDataset.csv")

In [8]:
dataco.printSchema()

root
 |-- Type: string (nullable = true)
 |-- Days for shipping (real): integer (nullable = true)
 |-- Days for shipment (scheduled): integer (nullable = true)
 |-- Benefit per order: double (nullable = true)
 |-- Sales per customer: double (nullable = true)
 |-- Delivery Status: string (nullable = true)
 |-- Late_delivery_risk: integer (nullable = true)
 |-- Category Id: integer (nullable = true)
 |-- Category Name: string (nullable = true)
 |-- Customer City: string (nullable = true)
 |-- Customer Country: string (nullable = true)
 |-- Customer Email: string (nullable = true)
 |-- Customer Fname: string (nullable = true)
 |-- Customer Id: integer (nullable = true)
 |-- Customer Lname: string (nullable = true)
 |-- Customer Password: string (nullable = true)
 |-- Customer Segment: string (nullable = true)
 |-- Customer State: string (nullable = true)
 |-- Customer Street: string (nullable = true)
 |-- Customer Zipcode: integer (nullable = true)
 |-- Department Id: integer (nullable = 

In [9]:
date_format = 'M/d/yyyy H:mm'

dataco = dataco.withColumn(
        "Order_datetime", 
        to_timestamp(col("order date (DateOrders)"), date_format)
    ).withColumn(
        "Shipping_datetime",
        to_timestamp(col("Shipping date (DateOrders)"), date_format)
    )
dataco.select(
    "order date (DateOrders)", 
    "order_datetime", 
    "Shipping date (DateOrders)", 
    "shipping_datetime"
).show(5, truncate=False)

+-----------------------+-------------------+--------------------------+-------------------+
|order date (DateOrders)|order_datetime     |Shipping date (DateOrders)|shipping_datetime  |
+-----------------------+-------------------+--------------------------+-------------------+
|1/31/2018 22:56        |2018-01-31 22:56:00|2/3/2018 22:56            |2018-02-03 22:56:00|
|1/13/2018 12:27        |2018-01-13 12:27:00|1/18/2018 12:27           |2018-01-18 12:27:00|
|1/13/2018 12:06        |2018-01-13 12:06:00|1/17/2018 12:06           |2018-01-17 12:06:00|
|1/13/2018 11:45        |2018-01-13 11:45:00|1/16/2018 11:45           |2018-01-16 11:45:00|
|1/13/2018 11:24        |2018-01-13 11:24:00|1/15/2018 11:24           |2018-01-15 11:24:00|
+-----------------------+-------------------+--------------------------+-------------------+
only showing top 5 rows


In [10]:
dataco = dataco.drop("order date (DateOrders)", "Shipping date (DateOrders)")

In [11]:
pd.set_option("display.max_colwidth", None)

description = spark.read.csv("DescriptionDataCoSupplyChain.csv", header=True, inferSchema=True)
description.limit(52).toPandas()

,FIELDS,DESCRIPTION
0,Type,: Type of transaction made
1,Days for shipping (real),: Actual shipping days of the purchased product
2,Days for shipment (scheduled),: Days of scheduled delivery of the purchased product
3,Benefit per order,: Earnings per order placed
4,Sales per customer,: Total sales per customer made per customer
5,Delivery Status,": Delivery status of orders: Advance shipping , Late delivery , Shipping canceled , Shipping on time"
6,Late_delivery_risk,": Categorical variable that indicates if sending is late (1), it is not late (0)."
7,Category Id,: Product category code
8,Category Name,: Description of the product category
9,Customer City,: City where the customer made the purchase


In [12]:
dataco.limit(3).toPandas()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,Customer Country,Customer Email,Customer Fname,Customer Id,Customer Lname,Customer Password,Customer Segment,Customer State,Customer Street,Customer Zipcode,Department Id,Department Name,Latitude,Longitude,Market,Order City,Order Country,Order Customer Id,Order Id,Order Item Cardprod Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,Shipping Mode,Order_datetime,Shipping_datetime
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Cally,20755,Holloway,XXXXXXXXX,Consumer,PR,5365 Noble Nectar Island,725,2,Fitness,18.251453,-66.037056,Pacific Asia,Bekasi,Indonesia,20755,77202,1360,13.110000,0.04,180517,327.75,0.29,1,327.75,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,NaN,1360,73,None,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,Standard Class,2018-01-31 22:56:00,2018-02-03 22:56:00
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Irene,19492,Luna,XXXXXXXXX,Consumer,PR,2679 Rustic Loop,725,2,Fitness,18.279451,-66.037064,Pacific Asia,Bikaner,India,19492,75939,1360,16.389999,0.05,179254,327.75,-0.80,1,327.75,311.359985,-249.089996,South Asia,Rajastán,PENDING,NaN,1360,73,None,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,Standard Class,2018-01-13 12:27:00,2018-01-18 12:27:00
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,EE. UU.,XXXXXXXXX,Gillian,19491,Maldonado,XXXXXXXXX,Consumer,CA,8510 Round Bear Gate,95125,2,Fitness,37.292233,-121.881279,Pacific Asia,Bikaner,India,19491,75938,1360,18.030001,0.06,179253,327.75,-0.80,1,327.75,309.720001,-247.779999,South Asia,Rajastán,CLOSED,NaN,1360,73,None,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,Standard Class,2018-01-13 12:06:00,2018-01-17 12:06:00


In [13]:
print("Number of rows:", dataco.count())

Number of rows: 180519


In [14]:
dataco = dataco.withColumnRenamed("Late_delivery_risk", "Is_late")

# Check Nulls

In [15]:
checkNull = dataco.select([sum(col(c).isNull().cast("int")).alias(c) for c in dataco.columns])
checkNull.toPandas().T

,0
Type,0
Days for shipping (real),0
Days for shipment (scheduled),0
Benefit per order,0
Sales per customer,0
Delivery Status,0
Is_late,0
Category Id,0
Category Name,0
Customer City,0


There are some null values in `"Customer Lname"` and `"Customer Zipcode"`. Also, there are a lot of nulls in `"Order Zipcode"` and `"Product Description"`.

# Data Cleaning

There are too many missing data on `"Order Zipcode"` and `"Product Description"`. In order to avoid bias, we have to drop these 2 columns.

In [16]:
dataco_clean = dataco.drop("Order Zipcode", "Product Description")

We can use `"Customer Id"` as a customer identifier, so we can drop `"Customer Fname"` and `"Customer Lname"`

In [17]:
dataco_clean = dataco_clean.drop("Customer Fname", "Customer Lname")

We can also use `"Category Name"` and `"Department Name"`, instead of their IDs.

In [18]:
dataco_clean = dataco_clean.drop("Category Id", "Department Id")

Check if the `"Customer Zipcode"` can be identified by `"Customer State"` and `"Customer Street"`.

In [19]:
null_zipcode = dataco_clean.filter(col("Customer Zipcode").isNull()).select("Customer State", "Customer Street").distinct()

null_zipcode.join(
    dataco_clean,
    on=["Customer State", "Customer Street"],
    how="inner"
).select("Customer State", "Customer Street", "Customer Zipcode").show()


+--------------+---------------+----------------+
|Customer State|Customer Street|Customer Zipcode|
+--------------+---------------+----------------+
|         95758|      Elk Grove|            NULL|
|         95758|      Elk Grove|            NULL|
|         91732|       El Monte|            NULL|
+--------------+---------------+----------------+



All missing values in `"Customer Zipcode"` cannot be imputed since no matching records with the same `"Customer State"` and `"Customer Street"` contain a valid zipcode. So, we need to delete the records to avoid false informations.

In [20]:
dataco_clean = dataco_clean.filter(col("Customer Zipcode").isNotNull())

In [21]:
checkNull = dataco_clean.select([sum(col(c).isNull().cast("int")).alias(c) for c in dataco_clean.columns])
checkNull.toPandas().T

,0
Type,0
Days for shipping (real),0
Days for shipment (scheduled),0
Benefit per order,0
Sales per customer,0
Delivery Status,0
Is_late,0
Category Name,0
Customer City,0
Customer Country,0


Before, we can see that `"Customer Email"` and `"Customer Password"` is totally masked to ensure privacy. We can delete those columns. Also, we don't need `"Product Image"` column.

In [22]:
dataco_clean = dataco_clean.drop("Customer Email", "Customer Password", "Product Image")

In [23]:
dataco_clean.limit(3).toPandas()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Is_late,Category Name,Customer City,Customer Country,Customer Id,Customer Segment,Customer State,Customer Street,Customer Zipcode,Department Name,Latitude,Longitude,Market,Order City,Order Country,Order Customer Id,Order Id,Order Item Cardprod Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Product Card Id,Product Category Id,Product Name,Product Price,Product Status,Shipping Mode,Order_datetime,Shipping_datetime
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,Sporting Goods,Caguas,Puerto Rico,20755,Consumer,PR,5365 Noble Nectar Island,725,Fitness,18.251453,-66.037056,Pacific Asia,Bekasi,Indonesia,20755,77202,1360,13.110000,0.04,180517,327.75,0.29,1,327.75,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,1360,73,Smart watch,327.75,0,Standard Class,2018-01-31 22:56:00,2018-02-03 22:56:00
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,Sporting Goods,Caguas,Puerto Rico,19492,Consumer,PR,2679 Rustic Loop,725,Fitness,18.279451,-66.037064,Pacific Asia,Bikaner,India,19492,75939,1360,16.389999,0.05,179254,327.75,-0.80,1,327.75,311.359985,-249.089996,South Asia,Rajastán,PENDING,1360,73,Smart watch,327.75,0,Standard Class,2018-01-13 12:27:00,2018-01-18 12:27:00
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,Sporting Goods,San Jose,EE. UU.,19491,Consumer,CA,8510 Round Bear Gate,95125,Fitness,37.292233,-121.881279,Pacific Asia,Bikaner,India,19491,75938,1360,18.030001,0.06,179253,327.75,-0.80,1,327.75,309.720001,-247.779999,South Asia,Rajastán,CLOSED,1360,73,Smart watch,327.75,0,Standard Class,2018-01-13 12:06:00,2018-01-17 12:06:00


Deleting duplicated data.

In [24]:
dataco_clean = dataco_clean.dropDuplicates()

In [25]:
total_rows = dataco_clean.count()
total_rows

180516

# Basic Data Analysis

In [26]:
num_cols = [col for col, dtype in dataco_clean.dtypes if dtype in ["int", "double", "bigint", "float", "decimal"]]
cat_cols = [col for col, dtype in dataco_clean.dtypes if dtype not in ["int", "double", "bigint", "float", "decimal"]]

print("Numeric columns: ", num_cols)
print("Categorical columns: ", cat_cols)

Numeric columns:  ['Days for shipping (real)', 'Days for shipment (scheduled)', 'Benefit per order', 'Sales per customer', 'Is_late', 'Customer Id', 'Customer Zipcode', 'Latitude', 'Longitude', 'Order Customer Id', 'Order Id', 'Order Item Cardprod Id', 'Order Item Discount', 'Order Item Discount Rate', 'Order Item Id', 'Order Item Product Price', 'Order Item Profit Ratio', 'Order Item Quantity', 'Sales', 'Order Item Total', 'Order Profit Per Order', 'Product Card Id', 'Product Category Id', 'Product Price', 'Product Status']
Categorical columns:  ['Type', 'Delivery Status', 'Category Name', 'Customer City', 'Customer Country', 'Customer Segment', 'Customer State', 'Customer Street', 'Department Name', 'Market', 'Order City', 'Order Country', 'Order Region', 'Order State', 'Order Status', 'Product Name', 'Shipping Mode', 'Order_datetime', 'Shipping_datetime']


### Numeric

In [27]:
dataco_clean.select(num_cols).describe().toPandas()

,summary,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Is_late,Customer Id,Customer Zipcode,Latitude,Longitude,Order Customer Id,Order Id,Order Item Cardprod Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Product Card Id,Product Category Id,Product Price,Product Status
0,count,180516,180516,180516,180516,180516,180516,180516,180516,180516,180516,180516,180516,180516,180516,180516,180516,180516,180516,180516,180516,180516,180516,180516,180516,180516
1,mean,3.497656717410091,2.931845376587117,21.974593354891514,183.10772341631034,0.5482948879877684,6691.23701500144,35921.12691395776,29.719830269997708,-84.91507399579824,6691.23701500144,36221.305097609074,692.4987591127656,20.664413362760104,0.10166749811797349,90258.59126060847,141.23129778282023,0.1206440425894033,2.1276562742360787,203.7718832866182,183.10772341631034,21.974593354891514,692.4987591127656,31.850855325843693,141.23129778282023,0.0
2,stddev,1.6237306336004638,1.3744485815187095,104.43429966618497,120.04387204981084,0.49766351664545455,4162.802257777325,37542.46112229251,9.813676205753216,21.432910654149463,4162.802257777325,21045.05638944893,336.4387732224458,21.80063284906583,0.07041453875098994,52110.77789894061,139.73227816999315,0.4667989832767063,1.4534562891346745,132.2730704681262,120.04387204981084,104.43429966618497,336.4387732224458,15.639497669885547,139.73227816999315,0.0
3,min,0,0,-4274.97998,7.489999771,0,1,603,-33.93755341,-158.0259857,1,1,19,0.0,0.0,1,9.989999771,-2.75,1,9.989999771,7.489999771,-4274.97998,19,2,9.989999771,0
4,max,6,4,911.7999878,1939.98999,1,20757,99205,48.78193283,115.2630768,20757,77204,1363,500.0,0.25,180519,1999.98999,0.5,5,1999.98999,1939.98999,911.7999878,1363,76,1999.98999,0


`"Is_late"` should be in categorical columns.

In [28]:
num_cols.remove("Is_late")
cat_cols.append("Is_late")

All of the `"Product Status"` is 0, indicating all products are available. Hence, it will not be useful for analysis.

In [29]:
dataco_clean.drop("Product Status")

DataFrame[Type: string, Days for shipping (real): int, Days for shipment (scheduled): int, Benefit per order: double, Sales per customer: double, Delivery Status: string, Is_late: int, Category Name: string, Customer City: string, Customer Country: string, Customer Id: int, Customer Segment: string, Customer State: string, Customer Street: string, Customer Zipcode: int, Department Name: string, Latitude: double, Longitude: double, Market: string, Order City: string, Order Country: string, Order Customer Id: int, Order Id: int, Order Item Cardprod Id: int, Order Item Discount: double, Order Item Discount Rate: double, Order Item Id: int, Order Item Product Price: double, Order Item Profit Ratio: double, Order Item Quantity: int, Sales: double, Order Item Total: double, Order Profit Per Order: double, Order Region: string, Order State: string, Order Status: string, Product Card Id: int, Product Category Id: int, Product Name: string, Product Price: double, Shipping Mode: string, Order_da

### Categorical

In [30]:
for col_name in cat_cols:
    dataco_clean.groupBy(col_name).count().orderBy(col("count").desc()).show(10)

+--------+-----+
|    Type|count|
+--------+-----+
|   DEBIT|69293|
|TRANSFER|49883|
| PAYMENT|41724|
|    CASH|19616|
+--------+-----+

+-----------------+-----+
|  Delivery Status|count|
+-----------------+-----+
|    Late delivery|98976|
| Advance shipping|41592|
| Shipping on time|32194|
|Shipping canceled| 7754|
+-----------------+-----+

+--------------------+-----+
|       Category Name|count|
+--------------------+-----+
|              Cleats|24551|
|      Men's Footwear|22246|
|     Women's Apparel|21035|
|Indoor/Outdoor Games|19298|
|             Fishing|17325|
|        Water Sports|15540|
|    Camping & Hiking|13729|
|    Cardio Equipment|12487|
|       Shop By Sport|10984|
|         Electronics| 3156|
+--------------------+-----+
only showing top 10 rows
+-------------+-----+
|Customer City|count|
+-------------+-----+
|       Caguas|66770|
|      Chicago| 3885|
|  Los Angeles| 3417|
|     Brooklyn| 3412|
|     New York| 1816|
| Philadelphia| 1577|
|        Bronx| 1500|
|  

# Advance Data Analysis

### Searching for Delivery Bottleneck

First, we count the percentages of late delivery.

In [31]:
is_late_group = dataco_clean.select("Is_late").groupBy("Is_late").count()

In [32]:
is_late_group.withColumn("percentage", round(col("count") / total_rows * 100, 2)).show()

+-------+-----+----------+
|Is_late|count|percentage|
+-------+-----+----------+
|      1|98976|     54.83|
|      0|81540|     45.17|
+-------+-----+----------+



More than half (54.83%) of our delivery is late! This is a big issue for our supply chain. If we solve this issue, it will have a big impact for the company.

Let's check which market have the highest late delivery percentage. Also, we will see the average days of lateness.

In [33]:
# Feature engineering for number of days of lateness
dataco_lateness = dataco_clean.withColumn("Lateness", col("Days for shipping (real)") - col("Days for shipment (scheduled)"))
dataco_lateness.select("Days for shipping (real)", "Days for shipment (scheduled)", "Lateness", "Is_late").show(8)

+------------------------+-----------------------------+--------+-------+
|Days for shipping (real)|Days for shipment (scheduled)|Lateness|Is_late|
+------------------------+-----------------------------+--------+-------+
|                       4|                            4|       0|      0|
|                       5|                            4|       1|      1|
|                       2|                            2|       0|      0|
|                       2|                            4|      -2|      0|
|                       6|                            2|       4|      1|
|                       3|                            4|      -1|      0|
|                       5|                            4|       1|      1|
|                       6|                            4|       2|      1|
+------------------------+-----------------------------+--------+-------+
only showing top 8 rows


Positive value on `"Lateness"` is telling the number of days the products is being delivered late.

In [34]:
dataco_lateness.groupBy("Market").agg(
    count("*").alias("Total_Orders"),
    sum("Is_late").alias("Total_late"),
    round(avg("Is_late")*100, 2).alias("Late_percentage"),
    round(
        sum(when(col("Is_late") == 1, col("Lateness"))) / sum("Is_late")
        , 2
    ).alias("Avg_lateness_IF_LATE")
).orderBy(col("Late_percentage").desc()).show()


+------------+------------+----------+---------------+--------------------+
|      Market|Total_Orders|Total_late|Late_percentage|Avg_lateness_IF_LATE|
+------------+------------+----------+---------------+--------------------+
|      Europe|       50250|     27742|          55.21|                1.62|
|Pacific Asia|       41259|     22712|          55.05|                1.62|
|        USCA|       25799|     14138|           54.8|                1.62|
|      Africa|       11614|      6340|          54.59|                 1.6|
|       LATAM|       51594|     28044|          54.36|                1.62|
+------------+------------+----------+---------------+--------------------+



Around half of the delivery on every market is late. We can assume that the lateness is not a regional problem, but more of a global or systemic problem. Next, we should check whether `"Shipping Mode"` have impact on the lateness.

In [35]:
dataco_lateness.groupBy("Shipping Mode").agg(
    count("*").alias("Total_Orders"),
    sum("Is_late").alias("Total_late"),
    round(avg("Is_late")*100, 2).alias("Late_percentage"),
    round(
        sum(when(col("Is_late") == 1, col("Lateness"))) / sum("Is_late")
        , 2
    ).alias("Avg_lateness_IF_LATE")
).orderBy(col("Late_percentage").desc()).show()


+--------------+------------+----------+---------------+--------------------+
| Shipping Mode|Total_Orders|Total_late|Late_percentage|Avg_lateness_IF_LATE|
+--------------+------------+----------+---------------+--------------------+
|   First Class|       27813|     26512|          95.32|                 1.0|
|  Second Class|       35216|     26987|          76.63|                 2.5|
|      Same Day|        9737|      4454|          45.74|                 1.0|
|Standard Class|      107750|     41023|          38.07|                1.51|
+--------------+------------+----------+---------------+--------------------+



The result shows that:
- Almost all of the "First Class" shipping is late!
- Although "Standard Class" generally pay a lower price, it is the most efficient shipping mode.
- When "First Class" and "Same Day" is late, on average they are only 1 day late. Meanwhile, when the "Second Class" is late, it can be 2.5 days late. More over, 76% of "Second Class" is late. This is a serious efficiency problem.
- Last, "Same Day" shipping mode that people hope for on emergency cases are late up to half of the total delivery. This is not acceptable.

Before investigating each `"Shipping Mode"`, we will make 1 more feature called "days_on_warehouse" from subtracting shipment date with order date. From this feature, we can investigate whether late delivery is caused by slow process on the warehouse or shipment.

In [36]:
dataco_shipping = dataco_lateness.withColumn(
    "days_on_warehouse",
    round(
        (col("shipping_datetime").cast("long") - col("order_datetime").cast("long")) / 3600 / 24
        ,2
    )
)

dataco_shipping.select("Shipping Mode", "shipping_datetime", "order_datetime", "days_on_warehouse").show(10)

+--------------+-------------------+-------------------+-----------------+
| Shipping Mode|  shipping_datetime|     order_datetime|days_on_warehouse|
+--------------+-------------------+-------------------+-----------------+
|Standard Class|2015-02-06 03:27:00|2015-02-02 03:27:00|              4.0|
|Standard Class|2017-05-27 06:08:00|2017-05-22 06:08:00|              5.0|
|  Second Class|2017-10-15 13:59:00|2017-10-13 13:59:00|              2.0|
|Standard Class|2018-01-14 09:08:00|2018-01-12 09:08:00|              2.0|
|  Second Class|2017-04-07 06:02:00|2017-04-01 06:02:00|              6.0|
|Standard Class|2017-08-14 09:21:00|2017-08-11 09:21:00|              3.0|
|Standard Class|2016-04-04 23:32:00|2016-03-30 23:32:00|              5.0|
|Standard Class|2015-09-28 11:53:00|2015-09-22 11:53:00|              6.0|
|  Second Class|2015-09-25 00:30:00|2015-09-23 00:30:00|              2.0|
|Standard Class|2017-01-02 02:08:00|2016-12-29 02:08:00|              4.0|
+--------------+---------

### Investigating "First Class" Shipping Mode

First let's check the difference between the estimated and real shipment days distribution on "First Class"

In [37]:
dataco_firstclass = dataco_shipping.filter(col("Shipping Mode") == "First Class")

In [38]:
dataco_firstclass.groupBy("Days for shipment (scheduled)", "Delivery Status").count().orderBy("count", ascending=False).show()
dataco_firstclass.groupBy("Days for shipping (real)", "Delivery Status").count().orderBy("count", ascending=False).show()

+-----------------------------+-----------------+-----+
|Days for shipment (scheduled)|  Delivery Status|count|
+-----------------------------+-----------------+-----+
|                            1|    Late delivery|26512|
|                            1|Shipping canceled| 1301|
+-----------------------------+-----------------+-----+

+------------------------+-----------------+-----+
|Days for shipping (real)|  Delivery Status|count|
+------------------------+-----------------+-----+
|                       2|    Late delivery|26512|
|                       2|Shipping canceled| 1301|
+------------------------+-----------------+-----+



In [39]:
dataco_firstclass.filter(
    (col("Is_late") == 0) & (col("Lateness") >= 0)
).select(
    "Days for shipment (scheduled)",
    "Days for shipping (real)",
    "Delivery Status",
    "Order status",
    "Is_late",
    "Lateness"
).distinct().show(5)

+-----------------------------+------------------------+-----------------+---------------+-------+--------+
|Days for shipment (scheduled)|Days for shipping (real)|  Delivery Status|   Order status|Is_late|Lateness|
+-----------------------------+------------------------+-----------------+---------------+-------+--------+
|                            1|                       2|Shipping canceled|       CANCELED|      0|       1|
|                            1|                       2|Shipping canceled|SUSPECTED_FRAUD|      0|       1|
+-----------------------------+------------------------+-----------------+---------------+-------+--------+



Turns out, none of the delivery on "First Class" is on time. 100% of the total delivery is late. This can be seen by:
- All scheduled days of shipment is 1 day. Meanwhile, the real days for shipping is 2 days.
- 95% late_percentage is caused by cancelled shipping because of cancelled order and fraud order.

Now, we want to see the distribution number of days the products are being processed on warehouse with "First Class" shipment.

In [40]:
dataco_firstclass.groupBy("days_on_warehouse").count().orderBy("count", ascending=False).show()

+-----------------+-----+
|days_on_warehouse|count|
+-----------------+-----+
|              2.0|27813|
+-----------------+-----+



The result shows that all of the "First Class" shipment products are in the warehouse for 2 days. Insights that we can get is that the products can actually be delivered on the same day as the orders, but it takes a longer time in the warehouse process.

### Investigating "Second Class" Shipping Mode

In [41]:
dataco_secondclass = dataco_shipping.filter(col("Shipping Mode") == "Second Class")

In [42]:
dataco_secondclass.groupBy("Days for shipment (scheduled)", "Delivery Status").count().orderBy("count", ascending=False).show()
dataco_secondclass.groupBy("Days for shipping (real)", "Delivery Status").count().orderBy("count", ascending=False).show()

+-----------------------------+-----------------+-----+
|Days for shipment (scheduled)|  Delivery Status|count|
+-----------------------------+-----------------+-----+
|                            2|    Late delivery|26987|
|                            2| Shipping on time| 6819|
|                            2|Shipping canceled| 1410|
+-----------------------------+-----------------+-----+

+------------------------+-----------------+-----+
|Days for shipping (real)|  Delivery Status|count|
+------------------------+-----------------+-----+
|                       2| Shipping on time| 6819|
|                       5|    Late delivery| 6772|
|                       3|    Late delivery| 6759|
|                       4|    Late delivery| 6759|
|                       6|    Late delivery| 6697|
|                       2|Shipping canceled|  319|
|                       3|Shipping canceled|  306|
|                       6|Shipping canceled|  286|
|                       5|Shipping canceled|  

These 2 tables shows that:
- The company is using a static 2-day promise for all orders, ignoring all variables.
- The fastest number of days for shipping is 2 day, so there is no way that the product will be delivered in advance before scheduled.

In [43]:
dataco_secondclass.groupBy("days_on_warehouse").count().orderBy("count", ascending=False).show()

+-----------------+-----+
|days_on_warehouse|count|
+-----------------+-----+
|              2.0| 7138|
|              3.0| 7065|
|              5.0| 7052|
|              6.0| 6983|
|              4.0| 6978|
+-----------------+-----+



The data shows an almost even distribution for processing times ranging from 2 to 6 days.

### Investigating "Same Day" Shipping Mode

In [44]:
dataco_sameday = dataco_shipping.filter(col("Shipping Mode") == "Same Day")

In [45]:
dataco_sameday.groupBy("Days for shipment (scheduled)", "Delivery Status").count().orderBy("count", ascending=False).show()
dataco_sameday.groupBy("Days for shipping (real)", "Delivery Status").count().orderBy("count", ascending=False).show()

+-----------------------------+-----------------+-----+
|Days for shipment (scheduled)|  Delivery Status|count|
+-----------------------------+-----------------+-----+
|                            0| Shipping on time| 4839|
|                            0|    Late delivery| 4454|
|                            0|Shipping canceled|  444|
+-----------------------------+-----------------+-----+

+------------------------+-----------------+-----+
|Days for shipping (real)|  Delivery Status|count|
+------------------------+-----------------+-----+
|                       0| Shipping on time| 4839|
|                       1|    Late delivery| 4454|
|                       0|Shipping canceled|  241|
|                       1|Shipping canceled|  203|
+------------------------+-----------------+-----+



This shipping mode is the hardest category to be on time. Customers that are paying for "Same Day" is gambling whether their product will arrive on time or not. We can see it from relatively same total number of products being delivered on time and late. But, if it is late, at most will only be late for 1 day.

### Investigating "Standard Class" Shipping Mode

In [46]:
dataco_standardclass = dataco_shipping.filter(col("Shipping Mode") == "Standard Class")

In [47]:
dataco_standardclass.groupBy("Days for shipment (scheduled)", "Delivery Status").count().orderBy("count", ascending=False).show()
dataco_standardclass.groupBy("Days for shipping (real)", "Delivery Status").count().orderBy("count", ascending=False).show()

+-----------------------------+-----------------+-----+
|Days for shipment (scheduled)|  Delivery Status|count|
+-----------------------------+-----------------+-----+
|                            4| Advance shipping|41592|
|                            4|    Late delivery|41023|
|                            4| Shipping on time|20536|
|                            4|Shipping canceled| 4599|
+-----------------------------+-----------------+-----+

+------------------------+-----------------+-----+
|Days for shipping (real)|  Delivery Status|count|
+------------------------+-----------------+-----+
|                       2| Advance shipping|20873|
|                       6|    Late delivery|20792|
|                       3| Advance shipping|20719|
|                       4| Shipping on time|20536|
|                       5|    Late delivery|20231|
|                       4|Shipping canceled|  997|
|                       3|Shipping canceled|  981|
|                       6|Shipping cancel

Again, the company is promising a static 4-day for their products to arrive on "Standard Class". Meanwhile, the real shipping vary from 2 to 6 days.

### Conclusion on Shipping Mode

The investigation reveals that Shipping Mode is the primary driver of delivery failures, but each mode fails in a distinct and symptomatic way. The root cause is a fundamental disconnect between a static, one-size-fits-all scheduling system and the dynamic realities of the logistics operation.

- **First Class**: A Systemic Failure \
    This premium service is broken by design. A hard-coded promise of 1-day delivery consistently clashes with a 2-day operational reality, making nearly every fulfilled order systematically late by exactly one day. The service promise is fundamentally misaligned with capability.

- **Second Class**: A Failure of Consistency \
    This service suffers from extreme variability. A rigid 2-day promise is applied to all orders, while actual delivery times range widely from 2 to 6 days. This indicates the scheduling algorithm completely ignores critical factors like destination, product type, or internal processing time.

- **Same Day**: A High-Stakes Gamble \
    This service operates at the absolute limit of its capacity. The extreme 0-day promise results in a nearly 50/50 success rate. Any minor friction in the fulfillment process causes a consistent 1-day delay. The service is, by definition, unreliable.

- **Standard Class**: The Unlikely Hero \
    Paradoxically, the cheapest and highest-volume service is the most reliable. This strongly suggests its delivery promise is the only one that is realistically aligned with the company's actual logistical performance.

In summary, the company's service level promises are inversely correlated with their reliability. The core issue is an unsophisticated scheduling system that needs to be replaced with a dynamic, data-driven model that sets delivery expectations based on real-world variables.

### Future Works

For future works, here are the list that can be done:
- **Analyze Key Factors**: Identify which variables (geography, product type, warehouse processing time) have the biggest impact on shipping delays.
- **Build a Predictive Model**: Use these factors to create a model that sets accurate, dynamic delivery dates for every order, replacing the current static system. Instead of a fixed number of days, the model will predict a realistic shipping time for each specific order based on key drivers
- **Create a Live Dashboard**: Develop a real-time dashboard to continuously monitor shipping performance and prevent future bottlenecks.